In [17]:
from gensim.models import Doc2Vec, ldamulticore
from gensim import corpora
from konlpy.tag import Okt, Komoran, Hannanum, Kkma, Mecab
from gensim.models.doc2vec import TaggedDocument
#mecab = Mecab(dicpath="C:/mecab/mecab-ko-dic")

# fasttext 실습

In [ ]:
from __future__ import print_function
from gensim.models import KeyedVectors

# Creating the model
ko_model = KeyedVectors.load_word2vec_format('wiki.ko.vec')

C:\Users\LG\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\LG\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# Getting the tokens 
words = []
for word in ko_model.vocab:
    words.append(word)

# Printing out number of tokens available
print("Number of Tokens: {}".format(len(words)))

# Printing out the dimension of a word vector 
print("Dimension of a word vector: {}".format(
    len(ko_model[words[0]])
))



In [ ]:
# Print out the vector of a word 
print("Vector components of a word: {}".format(
    ko_model[words[0]]
))

print(words[0])

## pre-trained 된 fasttext 모델로 단어간 유사도 검색을 할 수 있다.

In [ ]:
# Pick a word 
find_similar_to = '사랑'

# Finding out similar words [default= top 10]
for similar_word in ko_model.similar_by_word(find_similar_to):
    print("Word: {0}, Similarity: {1:.2f}".format(
        similar_word[0], similar_word[1]
    ))


In [ ]:
# Test words 
word_add = ['동물', '파충류']
word_sub = ['뱀']

# Word vector addition and subtraction 
for resultant_word in ko_model.most_similar(
    positive=word_add, negative=word_sub
):
    print("Word : {0} , Similarity: {1:.2f}".format(
        resultant_word[0], resultant_word[1]
    ))

# 시각화

## wi.ko.vec에 저장된 88만여개 단어 중 선두 300개 단어를 2차원 상에 맵핑하면 다음과 같은 그림이 보여짐

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc, rcParams
font_name = font_manager.FontProperties(fname="MALGUN.TTF").get_name()
rc('font', family=font_name)
rcParams.update({'figure.autolayout': True})
%matplotlib inline

# Limit number of tokens to be visualized
limit = 300
vector_dim = 300

# Getting tokens and vectors
words = []
embedding = np.array([])
i = 0
for word in ko_model.vocab:
    # Break the loop if limit exceeds 
    if i == limit: break

    # Getting token 
    words.append(word)

    # Appending the vectors 
    embedding = np.append(embedding, ko_model[word])

    i += 1

# Reshaping the embedding vector 
embedding = embedding.reshape(limit, vector_dim)


def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
    assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    plt.figure(figsize=(18, 18))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(label,
                 xy=(x, y),
                 xytext=(10, 4),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')
    plt.savefig(filename)


# Creating the tsne plot [Warning: will take time]
tsne = TSNE(perplexity=30.0, n_components=2, init='pca', n_iter=5000)

low_dim_embedding = tsne.fit_transform(embedding)

# Finally plotting and saving the fig 
plot_with_labels(low_dim_embedding, words)

In [ ]:
similarities = ko_model.wv.most_similar(positive=['동물', '파충류'], negative=['뱀'])
print(similarities)

not_matching = ko_model.wv.doesnt_match("아침 점심 저녁 된장국".split())
print(not_matching)

sim_score = ko_model.wv.similarity('컴퓨터', '인간')
print(sim_score)

sim_score = ko_model.wv.similarity('로봇', '인간')
print(sim_score)

sim_score = ko_model.wv.similarity('사랑해', '사랑의')
print(sim_score)

print(ko_model.wv.most_similar('전자'))

m = Mecab.Tagger()

## 필요한 함수

In [3]:
def get_tokenizer(tokenizer_name):
    if tokenizer_name == "komoran":
        tokenizer = Komoran()
    elif tokenizer_name == "okt":
        tokenizer = Okt()
    elif tokenizer_name == "mecab":
        tokenizer = Mecab()
    elif tokenizer_name == "hannanum":
        tokenizer = Hannanum()
    elif tokenizer_name == "kkma":
        tokenizer = Kkma()
    elif tokenizer_name == "khaiii":
        tokenizer = KhaiiiApi()
    else:
        tokenizer = Mecab()
    return tokenizer

class Doc2VecInput:

    def __init__(self, fname, tokenizer_name='hannanum'):
        self.fname = fname
        self.tokenizer = get_tokenizer(tokenizer_name)

    def __iter__(self):
        with open(self.fname, encoding='utf-8') as f:
            for line in f:
                try:
                    sentence, movie_id = line.strip().split("\u241E")
                    tokens = self.tokenizer.morphs(sentence)
                    tagged_doc = TaggedDocument(words=tokens, tags=['movie_%s' % movie_id])
                    yield tagged_doc
                except:
                    continue

## Doc2Vec

In [4]:
corpus_fname = "C:/work/BOAZ embeding/test.txt"
output_fname = "C:/work/BOAZ embeding/doc2vec.model"

In [5]:
corpus = Doc2VecInput(corpus_fname)

In [6]:
with open("C:/work/BOAZ embeding/test.txt", 'rt', encoding='UTF8') as f:
    print(f.readline())
    print("--------------------------------------------------------------------------------------------")
    print(f.readline())
    print("--------------------------------------------------------------------------------------------")
    print(f.readline())
    print("--------------------------------------------------------------------------------------------")
    print(f.readline())
    print("--------------------------------------------------------------------------------------------")
    count = 4
    for sentence in f:
        count+=1
    print("총 sentence의 개수 : {}개".format(count))

종합 평점은 4점 드립니다.␞92575

--------------------------------------------------------------------------------------------
원작이 칭송받는 이유는 웹툰 계 자체의 질적 저하가 심각하기 때문.  원작이나 영화나 별로인건 마찬가지.␞92575

--------------------------------------------------------------------------------------------
나름의  감동도 있고 안타까운 마음에 가슴도 먹먹  배우들의 연기가 good 김수현 최고~␞92575

--------------------------------------------------------------------------------------------
이런걸 돈주고 본 내자신이 후회스럽다 최악의 쓰레기 영화 김수현 밖에없는 저질 삼류영화␞92575

--------------------------------------------------------------------------------------------
총 sentence의 개수 : 5145개


모델에 사용된 감상평은 총 5145개

In [7]:
model = Doc2Vec(corpus, dm = 1, vector_size = 100)
model.save(output_fname)

dm이 1이면 PV-DM, 0이면 PV-DBOW  
vector_size 는 임베딩 벡터의 크기. 벡터 사이즈가 클수록 생성된 모델의 성능이 정교해지나 훈련 시간 및 메모리의 크기가 사이즈가 커진다는 단점이 있다. Glove 논문에 따르면 vector size가 커질수록 모델 성능이 좋아지는 것을 확인할 수 있습니다. vector 사이즈가 100까지 증가할 경우 가장 극적인 성능 향상을 보이고 그 이상의 사이즈일 경우 정확도 그래프가 완만하게 증가하는 것을 확인 할 수 있습니다.

In [54]:
import requests
from lxml import html
import random

class Doc2VecEvaluator:

    def __init__(self, model_fname="data/doc2vec.vecs", use_notebook=False):
        self.model = Doc2Vec.load(model_fname)
        self.doc2idx = {el:idx for idx, el in enumerate(self.model.docvecs.doctags.keys())}
        self.use_notebook = use_notebook

    def most_similar(self, movie_id, topn=10):
        similar_movies = self.model.docvecs.most_similar('movie_' + str(movie_id), topn=topn)
        for movie_id, score in similar_movies:
            print(self.get_movie_title(movie_id), score)

    def get_titles_in_corpus(self, n_sample=5):
        movie_ids = random.sample(self.model.docvecs.doctags.keys(), n_sample)
        return {movie_id: self.get_movie_title(movie_id) for movie_id in movie_ids}

    def get_movie_title(self, movie_id):
        url = 'http://movie.naver.com/movie/point/af/list.nhn?st=mcode&target=after&sword=%s' % movie_id.split("_")[1]
        resp = requests.get(url)
        root = html.fromstring(resp.text)
        try:
            title = root.xpath('//div[@class="choice_movie_info"]//h5//a/text()')[0]
        except:
            title = ""
        return title

In [55]:
model_fname="C:/work/BOAZ embeding/doc2vec.model"
model = Doc2VecEvaluator(model_fname) #모델 저장

In [56]:
model.get_titles_in_corpus(n_sample=3)

{'movie_100677': '타임 투 러브', 'movie_120042': '야간비행', 'movie_74533': '119 구조대'}

해당 리뷰에 맞는 영화 이름을 매칭 시킨 후(get_movie_title함수) 랜덤 영화 3개를 추출

In [57]:
model.most_similar(100677, topn = 3)

우리는 동물원을 샀다 0.9999254941940308
뜨거운 녀석들 0.9999253153800964
인디애니박스: 셀마의 단백질 커피 0.9999243021011353


100677번 영화(타임 투 러브)와 가장 유사한 영화 3개를 추출

## 5.3 잠재 디리클레 할당

In [14]:
corpus_fname = "C:/work/BOAZ embeding/corrected_ratings_corpus.txt"
documents, tokenized_corpus = [], []
tokenizer = get_tokenizer("hannanum")

In [15]:
with open(corpus_fname, 'r', encoding='utf-8') as f:
    for document in f:
        tokens = list(set(tokenizer.morphs(document.strip())))
        documents.append(document)
        tokenized_corpus.append(tokens)

dictionary = corpora.Dictionary(tokenized_corpus)
corpus = [dictionary.doc2bow(text) for text in tokenized_corpus]

In [18]:
LDA = ldamulticore.LdaMulticore(corpus, id2word = dictionary,
                               num_topics = 30,
                               workers = 4)

num_topics: 토픽수(k)

In [20]:
all_topics = LDA.get_document_topics(corpus,
                                    minimum_probability = 0.5,
                                    per_word_topics = False)

0.5미만의 토픽 분포는 무시한다

In [43]:
for doc_idx, topic in enumerate(all_topics[:5]):
    print(doc_idx, topic)

0 [(27, 0.6311326)]
1 [(7, 0.65424)]
2 [(20, 0.7805107)]
3 [(24, 0.87320346)]
4 [(7, 0.93079644)]


ex) 0번 문서는 30개의 토픽 중 27번째 토픽의 확률값이 가장 높다

In [47]:
output_fname = 'C:/work/BOAZ embeding/lda' #모델 저장
with open(output_fname + ".results", 'w', encoding = "utf-8") as f:
    for doc_idx, topic in enumerate(all_topics):
        if len(topic) == 1:
            # tuple 형태로 되어있는 데이터로 가져와서 나눠줌
            topic_id, prob = topic[0]
            f.writelines(documents[doc_idx].strip() + "\u241E" + ' '.join(tokenized_corpus[doc_idx]) + "\u241E" + str(topic_id) + "\u241E" + str(prob) + "\n")
LDA.save(output_fname + ".model")

In [40]:
from gensim.models import LdaModel
from collections import defaultdict

class LDAEvaluator:

    def __init__(self, model_path="./lda", tokenizer_name="hannanum"):
        self.tokenizer = get_tokenizer(tokenizer_name)
        self.all_topics = self.load_results(model_path + ".results")
        self.model = LdaModel.load(model_path + ".model")

    def load_results(self, results_fname):
        topic_dict = defaultdict(list)
        with open(results_fname, 'r', encoding='utf-8') as f:
            for line in f:
                sentence, _, topic_id, prob = line.strip().split("\u241E")
                topic_dict[int(topic_id)].append((sentence, float(prob)))
        for key in topic_dict.keys():
            topic_dict[key] = sorted(topic_dict[key], key=lambda x: x[1], reverse=True)
        return topic_dict

    def show_topic_docs(self, topic_id, topn=10):
        return self.all_topics[topic_id][:topn]

    def show_topic_words(self, topic_id, topn=10):
        return self.model.show_topic(topic_id, topn=topn)

    def show_new_document_topic(self, documents):
        tokenized_documents = [self.tokenizer.morphs(document) for document in documents]
        curr_corpus = [self.model.id2word.doc2bow(tokenized_document) for tokenized_document in tokenized_documents]
        topics = self.model.get_document_topics(curr_corpus, minimum_probability=0.5, per_word_topics=False)
        for doc_idx, topic in enumerate(topics):
            if len(topic) == 1:
                topic_id, prob = topic[0]
                print(documents[doc_idx], ", topic id:", str(topic_id), ", prob:", str(prob))
            else:
                print(documents[doc_idx], ", there is no dominant topic")

In [49]:
model = LDAEvaluator('./lda')

In [50]:
model.show_topic_docs(topic_id = 0) #해당 토픽의 확률 값이 가장 높은 문서를 출력

[('에릭은 당연히 나왔어야 했다. 완벽한 삶을 쫒는 이병헌, 허나 사람은 자기 인생이 가장 소중하고 제일 중요하다. 여기 나오는 모든 애들이 다 자기 잇속만 챙긴다. 에릭 또한, 자신의 복수를 위해서 이병헌이 뭔 짓을 하건 상관없이 죽인 것.이게 삶이다',
  0.9817925),
 ('오히려 평점 조작같은데.. 감독이 충무로 미움 삿을수도 있다는 생각까지 든다...한국영화 별로 좋아하진 않지만.. 이영화가 이렇게까지 평점 낮을만큼 재미없진 않다. 이정도 평점의 다른 영화같지않은 영화들과 비교할때 이 평점은 이해가 되질 않는다.',
  0.9816869),
 ('줄리아 로버츠의 웃음은 정말 보는 사람을 기쁘게 해준다. 웃음 하나로 기쁨과 슬픔을 표현할 수 있는 배우..전통을 깨고 여성의 새로운 삶을 살으라고 영화는 보여주는 데 영화에서는 전통을 완전히 벗어나야 한다는 것만을 강조하는 것 같아 아쉽다.',
  0.98140126),
 ('고등학교때 보고 지금에서 생각난 영화 참~그땐 눈물 콧물 흘리면서 봤는데짐 보면 나올라나.. 하긴 세월이 많이 흘렀지 아마 짐 시대하곤 스토리자체가 뻔하니깐 하지만 지금도 잊혀지지 않는 장면은 주인공이 공부하는 장면과하늘도 갈라놓지 못한 그들의 사랑',
  0.9808263),
 ('영화를 보는 사이사이에 아쉬움이 남기는 하지만、그나마 성은님께서 성은이 망극하게도 화끈하게 연기해 주어서 재미있게 잘 봤습니다。영화의 스토리로는 여타 영화와 다를바가 없지만 배슬기도 청순하게 연기 잘 했습니다。다음번에는 좀 더 용기내기를 바랍니다！',
  0.97880334),
 ('급할수록 돌아가라는 말이 있다. 요즘같이 급하게 사는 세상속에.이런영화를 본다면,,인생은 급한게 아니라, 느린세상이 더 행복하다는걸 느끼게 해준다..',
  0.9750976),
 ('스토리를 풀어나가기 힘든 주제임에도 침착하고 노골적으로 잘 표현한 영화라고 생각한다.. 실제로는 더 심각했다는 사실에 충격을 느낀다..',
  0.9738683),
 ('지구는 인간들만이

In [51]:
model.show_topic_words(topic_id = 0) 

[('이', 0.03160628),
 ('하', 0.02849265),
 ('.', 0.027684068),
 ('는', 0.025728267),
 ('ㄴ', 0.023062158),
 ('지', 0.020668844),
 ('을', 0.01825664),
 ('은', 0.017982338),
 ('ㄹ', 0.017367618),
 ('다', 0.017247446)]

In [52]:
model.show_new_document_topic(["너무 사랑스러운 영화", "인생을 말하는 영화"])

너무 사랑스러운 영화 , topic id: 21 , prob: 0.8388832
인생을 말하는 영화 , topic id: 13 , prob: 0.67638844


새로운 문서의 토픽을 확인했다. 문서를 형태소 분석한 뒤 이를 모델에 넣어 토픽을 추론하고, 가장 높은 확률값을 지니는 토픽 ID와 확률을 리턴.